In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [34]:
PROJECT_PATH = '../'
DATA_PATH = '/scratch/ceph/swei20/data/ae/dataset/test'
PCA_PATH = '/scratch/ceph/dobos/data/pfsspec/import/stellar/rbf/bosz_5000_full/pca/spectra.h5'
sys.path.insert(0, PROJECT_PATH)

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
import tensorflow as tf
# tf.enable_v2_behavior()
gpus = tf.config.list_physical_devices('GPU') 
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Loading Data

In [35]:
with h5py.File(PCA_PATH, 'r') as f:
    flux = f['flux'][()]

In [39]:
def prepro_pca(flux, dim=300):
    flux0 = flux.reshape(-1, 1000)[:, :dim]    
    flux_clean = flux0[~np.isnan(flux0[:,0])]    
    print(flux_clean.shape, flux0.shape)
    print('min', flux_clean.min(), 'max', flux_clean.max(),'mean', flux_clean.mean(),'std', flux_clean.std() )
    return flux_clean

In [40]:
flux1 = prepro_pca(flux, dim=300)

(116614, 300) (243936, 300)
min -184.9254058126797 max 145.24091665337806 mean -0.10353327716635463 std 2.4000601615265493


In [155]:
SAVE_PATH = os.path.join(DATA_PATH, 'pca.h5')

In [156]:
with h5py.File(SAVE_PATH, 'w') as f:
    f.create_dataset('flux300', data = flux1)
#     f.create_dataset('flux10kmm', data = flux10kmm)

In [27]:
with h5py.File(fn, 'r') as f:
    flux10kmm = f['flux10kmm'][()]
#     flux10kmm = f['flux10kmm'][()]

## AE

In [265]:
from ae.pipeline.simple_ae_pipeline import SimpleAEPipeline

In [267]:
sp=SimpleAEPipeline()

In [292]:
config_dict = {
    "data": {
      "dir": "/scratch/ceph/swei20/data/ae/dataset/test",
      "filename": "pca.h5",
      "train": "flux300",
      "test": None
    },
    "model":{
      "name": "SimpleAE",
      "input_dim": 300,
      "latent_dim": 5,
      "hidden_dims": [128, 32],
      "dropout": 0.2,
      "lr": 0.01,
      "reg1": 0.0,
      "loss": "mae",
      "opt": "adam"
    },
    "trainer":{
      "epoch": 20,
      "batch_size": 32,
      "validation_split":0.1,
      "verbose_training": 1
    },
    "callbacks":{
        "checkpoint_monitor": "val_loss",
        "checkpoint_mode": "min"
    }
}
config = DotMap(config_dict)

In [276]:
vmax = np.max(abs(flux1))

In [277]:
vmax

184.9254058126797

In [290]:
flux_norm = flux1 / vmax 

In [288]:
flux_norm.min(), flux_norm.max()

(0.0, 0.8927013598134264)

In [293]:
data = flux_norm
sp.run_step_model(config, (data,data))

[300, 128, 32, 5]
Epoch 1/20
3280/3280 [==============================] - 9s 3ms/step - loss: 0.0015 - acc: 0.2341 - val_loss: 0.0018 - val_acc: 0.0875
Epoch 2/20
3280/3280 [==============================] - 8s 2ms/step - loss: 0.0015 - acc: 0.2308 - val_loss: 0.0016 - val_acc: 0.0941
Epoch 3/20
3280/3280 [==============================] - 8s 2ms/step - loss: 0.0014 - acc: 0.2405 - val_loss: 0.0017 - val_acc: 0.0803
Epoch 4/20
3280/3280 [==============================] - 8s 2ms/step - loss: 0.0014 - acc: 0.2348 - val_loss: 0.0016 - val_acc: 0.0657
Epoch 5/20
3280/3280 [==============================] - 8s 2ms/step - loss: 7.8994e-04 - acc: 0.3873 - val_loss: 9.9193e-04 - val_acc: 0.1979
Epoch 6/20
3280/3280 [==============================] - 8s 2ms/step - loss: 7.4429e-04 - acc: 0.4462 - val_loss: 0.0010 - val_acc: 0.2947
Epoch 7/20
3280/3280 [==============================] - 8s 2ms/step - loss: 7.2367e-04 - acc: 0.4596 - val_loss: 0.0010 - val_acc: 0.2404
Epoch 8/20
3280/3280 [======

In [259]:
# cmd = "main --config /home/swei20/AE/configs/ae/train/pca_config.json --lr 2"
# sys.argv = cmd.split()
# sp=SimpleAEPipeline()
# sp.prepare()
# sp.run()

Exception: arg epoch not specified in command line